In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk

# Lenguaje Claro - UPE
Lucas Pecina

09/01/21

---
### Objetivo
Comparar distintos textos (legales, judiciales, etc) para ver la complejidad que tiene ese texto en cuanto a palabras dificiles (poco frecuentes en el lenguaje cotidiano).

### Resumen
El lenguaje legal/judicial suele ser complicado de entender para la gente que no se dedica a temas relacionados debido a la complejidad de las palabras utilizadas y de su gramatica. El analisis de complejidad gramatical de los textos requiere la utilizacion de tecnicas muy sofisticadas y que no dan buenos resultados, por lo tanto, descartamos este tipo de estudio por el momento.

Comenzamos solo analizando las **palabras** de los textos y creamos un **indice de complejidad** que mide la cantidad y frecuencia de palabras "complejas", que son de poca frecuencia en el lenguaje cotidiano. 

No contaran para el calculo aquellas palabras que por su naturaleza tecnica juridica requiera de su uso obligatorio. Para dicha tarea, se elaborara a mano una lista de las palabras en cuestion.

### Fuentes de datos
- Textos a analizar: seran cargados como archivos de texto
 - Codigo Procesal Civil y Comercial actual http://biblioteca.asesoria.gba.gov.ar/redirect.php?id=2121 esta en un archivo CPCC
 - Codigo Procesal Penal actual http://servicios.infoleg.gob.ar/infolegInternet/anexos/0-4999/383/texact.htm 
- Bolsa de palabras de frecuencia del lenguaje español http://corpus.rae.es/lfrecuencias.html. Usando las 300mil palabras mas frecuentes del español: en datos/

### Pasos
1. Corpus de palabras frecuentes
 - Armar dataframe con frecuencias
 - Calcular complejidad de palabras
2. Cargar textos y preprocesarlo
 - Tokenizar las palabras
 - Limpiar errores
 - Remover stopwords


---
### Indice de complejidad del lenguaje
$$Complejidad_t = \frac{1}{P}\sum_p^P{Complejidad_p}$$

- t = texto a analizar
- p = lista de palabras en el texto t (pueden estar repetidas)
- P = cantidad de palabras en el texto t (todas)

$$Complejidad_p = f(FrecuenciaBolsa_p)$$
> **DETERMINAR EL INDICE DE COMPLEJIDAD DE LAS PALABRAS**. 
Ver la distribucion de frecuencias (despues de remover stopwords). Ver si hacerlo por frecuencia de palabras o ranking.
Puede ser el ranking o el inverso de las frecuencias.

$$Complejidad_p = \frac{1}{FrecuenciaBolsa_p}$$

o

$$Complejidad_p = RankingBolsa_p$$
 

Corpus de palabras frecuentes del español
---



In [175]:
# cargo datos
corpus = pd.read_excel('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CREA_total.xlsx')
corpus = corpus.dropna()
corpus['Palabra'] = corpus.Palabra.astype('str').str.strip()
corpus = corpus[corpus.Frecuencia_norm>0.00]

# reemplazo caracteres mal cargados
reemplazar = {'·':'á','È':'é','Ò':'ñ','˙':'ú','¸':'ü','Ì':'í','Û':'ó'}

for k,v in reemplazar.items():
    corpus['Palabra'] = corpus.Palabra.str.replace(k,v)
corpus

,Orden,Palabra,Frecuencia,Frecuencia_norm
0,1,de,9999518,65545.55
1,2,la,6277560,41148.59
2,3,que,4681839,30688.85
3,4,el,4569652,29953.48
4,5,en,4234281,27755.16
...,...,...,...,...
423729,423730,zyscovich,2,0.01
423730,423731,zyx,2,0.01
423731,423732,zzz,2,0.01
423732,423733,zzzzzzzzzzz,2,0.01


In [176]:
corpus.Frecuencia_norm.sum()

995706.050000001

In [177]:
# calculo complejidad
corpus['complejidad'] = 1/corpus.Frecuencia_norm
corpus

,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad
0,1,de,9999518,65545.55,0.000015
1,2,la,6277560,41148.59,0.000024
2,3,que,4681839,30688.85,0.000033
3,4,el,4569652,29953.48,0.000033
4,5,en,4234281,27755.16,0.000036
...,...,...,...,...,...
423729,423730,zyscovich,2,0.01,100.000000
423730,423731,zyx,2,0.01,100.000000
423731,423732,zzz,2,0.01,100.000000
423732,423733,zzzzzzzzzzz,2,0.01,100.000000


Procesamiento de textos a analizar
---


In [186]:
# funcion para el preprocesamiento de los textos a cargar

def limpieza_textos(raw,corpus):
    tokens = nltk.word_tokenize(raw)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    print(f'El texto original tiene {len(tokens)} palabras.')
    print(f'De esas, hay {len(np.unique(np.array(tokens)))} unicas')
    
    # hay muchos que finalizan en "are", 'ere'. Mencionare, juzgare, interpusiere, etc.
    # a esos los saco
    tokens = [t for t in tokens if ('are' not in t) & ('ere' not in t)]
    print(f'Despues de sacarle las palabras que terminan en ARE o ERE, quedan {len(tokens)} palabras.')
    print(f'Ahora hay {len(np.unique(np.array(tokens)))} unicas')
    
    # veo las palabras del texto que no estan en el corpus y las agrego al corpus para que cuenten
    no_estan = [t for t in np.unique(np.array(tokens)) if t not in corpus.Palabra.values]
    print(f'Hay {len(no_estan)} parabras unicas que no aparecen en el corpus. Las agrego con 100 de complejidad')
    
    # ahora quedaron solo las que son muy raras o las que estan sin tildes por los titulos
    '''Ahora hay que agregar todas estas palabras al df de corpus y agregarle
    el valor maximo = 100 '''

    no_estan_df = pd.DataFrame({'Palabra':no_estan, 'complejidad':[100.0]*len(no_estan)})
    print('no estan df')
    print(no_estan_df)
    # concat
    corpus_ag = pd.concat([corpus[['Palabra','complejidad']],no_estan_df]).reset_index()
    
    return(tokens,corpus_ag)    

In [276]:
# funcion para calcular frecuencias de palabras en texto y calculo el indice_complejidad

def crear_indice_complejidad(tokens,corpus,nombre_texto,comunes_sacar):
    '''1 crear df de palabras de nombre_texto y las frecuencias
       2 joinear y calcular'''

    tokens_freq = pd.DataFrame.from_dict(nltk.FreqDist(tokens), orient='index').reset_index()
    tokens_freq.columns = ['Palabra','freq']
    #tokens_freq['freq_relativa'] = tokens_freq.freq/tokens_freq.freq.sum()

    print('tokens_freq')
    print(tokens_freq)
    
    # joinear
    tokens_complejidad = corpus.iloc[comunes_sacar:,:].merge(tokens_freq, left_on='Palabra',right_on='Palabra').reset_index(drop=True)
    
    # calculo indice_complejidad
    tokens_complejidad['indice_complejidad'] = (tokens_complejidad.complejidad * tokens_complejidad.freq)/tokens_complejidad.freq.sum()
    print(f'la cantidad de palabras que coincidieron en el corpus es de {len(tokens_complejidad)}')
    
    indice_complejidad = round(sum(tokens_complejidad.indice_complejidad),5)
    print(f'INDICE DE COMPLEJIDAD DE {nombre_texto}: {indice_complejidad}')
    
    return(tokens_complejidad, indice_complejidad)

In [277]:
# funcion para el preprocesamiento de los textos a cargar

def limpieza_textos_simple(raw, sacar_ere_ese=False):
    '''Considera como errores/cosas extremas a las palabras que estan
    en el texto pero no en el corpus'''
    tokens = nltk.word_tokenize(raw)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    
    if sacar_ere_ese:
        # saco a los que finalizan en "are", 'ere'. Mencionare, juzgare, interpusiere, etc.
        tokens = [t for t in tokens if ('are' not in t) & ('ere' not in t)]
        print('Luego de sacar las que tienen ARE o ERE (como conociere)')
    
    print(f'El texto original tiene {len(tokens)} palabras.')
    print(f'De esas, hay {len(np.unique(np.array(tokens)))} unicas')

    return(tokens)

In [262]:
comunes_sacar = 0
sacar_ere_ese = False

CPCC
---

In [278]:
cpcc_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPCC.txt')
cpcc_raw = cpcc_f.read()
cpcc_tokens = limpieza_textos_simple(cpcc_raw, sacar_ere_ese)
cpcc_tokens, cpcc_indice = crear_indice_complejidad(cpcc_tokens,corpus,'CPCC actual',comunes_sacar)
cpcc_tokens.sort_values('indice_complejidad',ascending=False).head(10)

El texto original tiene 62785 palabras.
De esas, hay 5627 unicas
tokens_freq
            Palabra  freq
0             libro    23
1                 i    32
2     disposiciones    45
3         generales    16
4            título    70
...             ...   ...
5622       cúmplase     1
5623    comuníquese     1
5624     publíquese     1
5625           dese     1
5626      archívese     1

[5627 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 5359
INDICE DE COMPLEJIDAD DE CPCC actual: 0.81736


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
5210,258917,correspondiere,4,0.02,50.0,27,0.021669
5303,350809,compareciere,2,0.01,100.0,13,0.020866
5257,297261,devolutivo,3,0.01,100.0,13,0.020866
5202,253597,apelable,4,0.02,50.0,23,0.018459
5232,267522,irrecurrible,4,0.02,50.0,13,0.010433
5273,310387,litispendencia,3,0.01,100.0,6,0.009631
5294,332839,admitiere,2,0.01,100.0,6,0.009631
5203,253598,apelante,4,0.02,50.0,10,0.008026
5336,392124,notificador,2,0.01,100.0,5,0.008026
5259,297747,dispusiere,3,0.01,100.0,5,0.008026


CPP
---

In [279]:
cpp_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPP.txt')
cpp_raw = cpp_f.read()

cpp_tokens = limpieza_textos_simple(cpp_raw, sacar_ere_ese)
cpp_tokens, cpp_indice = crear_indice_complejidad(cpp_tokens,corpus,'CPP actual',comunes_sacar)
cpp_tokens.sort_values('indice_complejidad',ascending=False).head(10)

El texto original tiene 44432 palabras.
De esas, hay 4445 unicas
tokens_freq
           Palabra  freq
0           codigo    11
1         procesal    29
2            penal   101
3              ley   163
4              ver     2
...            ...   ...
4440  permanecerán     1
4441       entrará     1
4442     efectuada     1
4443       reforma     1
4444       órganos     1

[4445 rows x 2 columns]
la cantidad de palabras que coincidieron en el corpus es de 4357
INDICE DE COMPLEJIDAD DE CPP actual: 0.47422


,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad,freq,indice_complejidad
4340,381324,labrará,2,0.01,100.000000,7,0.015797
4329,326089,supiere,3,0.01,100.000000,7,0.015797
4285,253597,apelable,4,0.02,50.000000,11,0.012412
4336,350809,compareciere,2,0.01,100.000000,4,0.009027
4303,267522,irrecurrible,4,0.02,50.000000,7,0.007899
4292,258917,correspondiere,4,0.02,50.000000,7,0.007899
4349,401415,quáter,2,0.01,100.000000,3,0.006770
4334,332839,admitiere,2,0.01,100.000000,3,0.006770
4274,237599,excusación,5,0.03,33.333333,8,0.006018
4253,217807,encontrare,6,0.03,33.333333,7,0.005266


Comparacion
---

In [280]:
print(f'La tasa de complejidad de uno sobre el otro es de: \n{round(cpcc_indice/cpp_indice, 2)} ({round((cpcc_indice-cpp_indice)/cpp_indice*100, 1)}% mas complejo)')

La tasa de complejidad de uno sobre el otro es de: 
1.72 (72.4% mas complejo)
